In [4]:
import numpy as np
from PIL import Image
import glob

In [5]:
folder = ["apple", "banana"]

# 画像とラベルを格納するための配列を準備
X = []
Y = []

#ファイルオープン～が配列生成までを関数化
#画像サイズは50x50に強制的に圧縮
def img(x):
    y = Image.open(x)
    y = y.convert("RGB")
    y = y.resize((50, 50))
    y = np.asarray(y)
    return y

#画像の読み込み処理　インデックスを取得するためにenumerateを使用
#folder名に一致するフォルダの中身を1次元配列かしながらXに格納
#同時にYにはインデックス(今回はaplleフォルダが0,bananaフォルダが1)がXと1:1対応で格納される

for index, name in enumerate(folder):
    dir = "./" + name
    files = glob.glob(dir + "/*.png")
    for i, file in enumerate(files):
        image = img(file)
        X.append(image.flatten())
        Y.append(index)

X = np.array(X) / 255
Y = np.array(Y)

from keras.utils import np_utils
#Kerasで使うためにYの値をバイナリに変換
Y = np_utils.to_categorical(Y, 2)

from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten

#モデルの構築
# とりあえず全結合のやつをつくってみる。一層目出力ノード数は適当
model = Sequential([Dense(512, input_dim=7500), Activation("relu"), Dense(2), Activation("softmax")])
#model = Sequential()
#model.add(Dense(512, activation='relu', input_shape=(7500,)))
#model.add(Dense(2, activation="softmax"))


#モデルのコンパイル
#損失関数：交差エントロピー,最適化関数：sgb, 評価関数：正解率(acc)
model.compile(
    loss="categorical_crossentropy",
    optimizer="sgd",
    metrics=["accuracy"],
)
#学習。バッチサイズ:5, ログ出力:ブログレスバー,反復数:20,検証データの割合:0.3
hist = model.fit(
    X, Y,
    batch_size=5,
    verbose=1,
    epochs=20,
    validation_split=0.3,
)


Using TensorFlow backend.
Train on 153 samples, validate on 66 samples
Epoch 1/20
153/153 [==============================] - 7s 48ms/step - loss: 1.0924 - acc: 0.7320 - val_loss: 1.0462 - val_acc: 0.2576
Epoch 2/20
153/153 [==============================] - 2s 15ms/step - loss: 0.3734 - acc: 0.8235 - val_loss: 1.7567 - val_acc: 0.1364
Epoch 3/20
153/153 [==============================] - 2s 14ms/step - loss: 0.1910 - acc: 0.9412 - val_loss: 0.1353 - val_acc: 0.9848
Epoch 4/20
153/153 [==============================] - 2s 14ms/step - loss: 0.2513 - acc: 0.9020 - val_loss: 0.2435 - val_acc: 0.9697
Epoch 5/20
153/153 [==============================] - 2s 14ms/step - loss: 0.1187 - acc: 0.9739 - val_loss: 1.7779 - val_acc: 0.2727
Epoch 6/20
153/153 [==============================] - 2s 14ms/step - loss: 0.1142 - acc: 0.9542 - val_loss: 0.1682 - val_acc: 0.9697
Epoch 7/20
153/153 [==============================] - 2s 14ms/step - loss: 0.0650 - acc: 0.9739 - val_loss: 2.0678 - val_acc: 0.242

In [7]:
Xt = []
testimg = img("ばなな.jpg")


Xt.append(testimg.flatten())
Xt = np.array(Xt)/255

#判定
result = model.predict_classes(Xt)

#判定結果
if result == 0:
    print("apple")
else:
    print("banana")

banana


In [9]:
#modelの保存の重みを保存
model.save('applebanana.hdf5')

#保存したモデルをロードする
#from keras.models import load_model
# model = keras.models.load_model('applebanana.hdf5')